In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [19]:
import json
from pathlib import Path
from typing import List

import dataclass_array as dca
import matplotlib.pyplot as plt
import mitsuba as mi
import numpy as np
from PIL import Image
import pycolmap
import pyrender
import trimesh
import visu3d as v3d

import burybarrel.colmap_util as cutil
from burybarrel.image import render_v3d

In [ ]:
reconstr_path = Path("/scratch/jeyan/barreldata/results/barrelddt1/colmap-out/0")
foundpose_res_path = Path("/scratch/jeyan/foundpose/output_barrelddt1/inference/estimated-poses.json")
obj_path = Path("/scratch/jeyan/barreldata/models3d/barrelsingle-scaled.ply")

reconstruction = pycolmap.Reconstruction(reconstr_path)
print(reconstruction.summary())

In [ ]:
p, c = cutil.get_pc(reconstruction)
pts3d = v3d.Point3d(p=p, rgb=c)
cams = cutil.get_cams_v3d(reconstruction)
v3d.make_fig([cams, pts3d])

In [6]:
with open(foundpose_res_path, "rt") as f:
    foundpose_res = json.load(f)
foundpose_res = list(filter(lambda x: x["hypothesis_id"] == "0", foundpose_res))

In [ ]:
foundpose_res[9]

In [8]:
transforms = []
for res in foundpose_res:
    R = res["R"]
    t = res["t"]
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = np.reshape(t, -1)
    transforms.append(T)
transforms = np.array(transforms)

In [134]:
mesh = trimesh.load(obj_path)
vtxs = np.array(mesh.vertices)
scalefactor = 0.2
scaleT = v3d.Transform.from_matrix(np.eye(4) * scalefactor)

In [ ]:
tofig = []
barrels_trf = []
tofig.extend([pts3d, cams])
for i, cam in enumerate(cams):
    cam2world = cam.world_from_cam
    obj2cam = v3d.Transform.from_matrix(transforms[i])
    rgb = np.zeros_like(vtxs, dtype=np.uint8)
    rgb[:, 0] = 255
    pts = v3d.Point3d(p=vtxs, rgb=rgb)
    pts = cam2world @ obj2cam @ pts
    tofig.extend([pts])
v3d.make_fig(*tofig)

In [ ]:
tofig = []
camscaled = cams.replace(world_from_cam=cams.world_from_cam.replace(t=cams.world_from_cam.t * scalefactor))
sceneptsscaled = pts3d.apply_transform(scaleT)
barrels_trf = []
tofig.extend([sceneptsscaled, camscaled])
for i, cam in enumerate(camscaled):
    cam2world = cam.world_from_cam
    obj2cam = v3d.Transform.from_matrix(transforms[i])
    rgb = np.zeros_like(vtxs, dtype=np.uint8)
    rgb[:, 0] = 255
    pts = v3d.Point3d(p=vtxs, rgb=rgb)
    pts = cam2world @ obj2cam @ pts
    barrels_trf.append(pts)
    tofig.extend([pts])
v3d.make_fig(*tofig)

In [ ]:
i = 9
v3d.make_fig(camscaled[i], sceneptsscaled, barrels_trf[i])

In [ ]:
i = 9
Image.fromarray(render_v3d(camscaled[i], dca.concat([sceneptsscaled, barrels_trf[i]]), radius=2))

In [ ]:
mesh = trimesh.load(obj_path)
vtxs = np.array(mesh.vertices)
stuff = []
stuff.append(pts3d)

i = 9
cam = cams[i]
cam2world = cam.world_from_cam
obj2cam = v3d.Transform.from_matrix(transforms[i])
rgb = np.zeros_like(vtxs, dtype=np.uint8)
rgb[:, 0] = 255
pts = v3d.Point3d(p=vtxs, rgb=rgb)
pts = cam2world @ obj2cam @ pts
stuff.extend([pts, cam])
Image.fromarray(render_v3d(cam, dca.concat([pts3d, pts]), radius=2))
# Image.fromarray(render_v3d(cam, dca.concat([pts3d]), radius=2))